In [2]:
import numpy as np
import pandas as pd
import matplotlib

train_df = pd.read_csv('../Train.csv')
test_df = pd.read_csv('../Test.csv')
seed = 7 #Set random seed for reproducability

In [3]:
train_df.head(n=2)


,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
train_df.describe()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
count,42000.000000,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,...,42000.000000,42000.000000,42000.000000,42000.00000,42000.000000,42000.000000,42000.0,42000.0,42000.0,42000.0
mean,4.456643,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.219286,0.117095,0.059024,0.02019,0.017238,0.002857,0.0,0.0,0.0,0.0
std,2.887730,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.312890,4.633819,3.274488,1.75987,1.894498,0.414264,0.0,0.0,0.0,0.0
min,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
25%,2.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
50%,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
75%,7.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
max,9.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,254.000000,254.000000,253.000000,253.00000,254.000000,62.000000,0.0,0.0,0.0,0.0


In [4]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

X = train_df.iloc[:,1:]
y = train_df.iloc[:,0]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.98, random_state = seed) #Test size set at beginning

In [11]:
clf = xgb.XGBClassifier(objective = 'multi:softmax')
clf.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [12]:
y_pred = clf.predict(X_test)

In [13]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

def printresults(Y_test, Y_pred):
    cm = confusion_matrix(Y_test, Y_pred)
    accuracy = accuracy_score(Y_test, Y_pred)
    print("Statistics summary\n-------------------------------------------------------------\n")
    print("Accuracy: {0}%\n".format(accuracy))
    print("Confusion matrix:")
    print(cm)
printresults(y_test, y_pred)

Statistics summary
-------------------------------------------------------------

Accuracy: 0.8587949465500486%

Confusion matrix:
[[3810    0   12   49   19   50   41   27   30    5]
 [   1 4441   14   30    8   18   25   10   10    7]
 [ 158  103 3251  105  114   43  118   82   87   40]
 [  56   36   95 3425   15  338   44   66  118   77]
 [  11   23   54   21 3316   34   32   31   35  435]
 [  70   53   11  144   42 3161   94   26   67   50]
 [  49   17   67    9   64  114 3660   16   50    3]
 [  17   74   82   43   53   34    3 3847   13  143]
 [  15  241   84  205   69  170   57   25 2939  194]
 [  29   41   44  106  122   38    4  193   40 3498]]


To get an image of the xgboost results scaling with the amount of data we'll run a test on 0.25, 50, 75 and 100% of the remaining training data after removing 20% for our private test set

In [23]:
#Reset X and Y
X = train_df.iloc[:,1:]
y = train_df.iloc[:,0]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = seed)

def create_model(training_size):
    Test_size = 1-training_size
    print("Training model with data size at: {0}%.".format(training_size))
    x, x_ignore, y, y_ignore = train_test_split(X_train, y_train, test_size = Test_size, random_state = seed)
    clf = xgb.XGBClassifier(objective = 'multi:softmax')
    clf.fit(x, y)
    print("Training finished.")
    return clf
    
def frange(start, stop, step):
    i = start
    while i <= stop:
        yield i
        i+=step

In [24]:
#Commented out as function has served its purpose
for i in frange(0.25, 1, 0.25) :
    #clf = create_model(i)
    #y_pred = clf.predict(X_test)
    #printresults(y_test, y_pred)

SyntaxError: unexpected EOF while parsing (<ipython-input-24-7d588cd62eb1>, line 5)

In [25]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1), copy=True)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from sklearn.svm import SVC


def create_svm_model(training_size):
    Test_size = 1-training_size
    print("Training model with data size at: {0}%.".format(training_size))
    X, x_ignore, y, y_ignore = train_test_split(X_train, y_train, test_size = Test_size, random_state = seed)
    clf = SVC(C = 1, kernel = 'rbf', gamma = 'auto')
    clf.fit(X, y)
    print("Training finished.")
    return clf

for i in frange(0.001, 0.02, 0.002):
    clf = create_svm_model(i)
    y_pred = clf.predict(X_test)
    printresults(y_test, y_pred)
    
for i in frange(0.25, 1, 0.25):
    clf = create_svm_model(i)
    y_pred = clf.predict(X_test)
    printresults(y_test, y_pred)

Training model with data size at: 0.001%.
Training finished.
Statistics summary
-------------------------------------------------------------

Accuracy: 0.10166666666666667%

Confusion matrix:
[[  9   0   0   0   0   0   0   0   0 824]
 [  0   0   0   0   0   0   0   0   0 958]
 [  0   0   0   0   0   0   0   0   0 815]
 [  0   0   0   0   0   0   0   0   0 879]
 [  0   0   0   0   0   0   0   0   0 820]
 [  0   0   0   0   0   0   0   0   0 732]
 [  0   0   0   0   0   0   0   0   0 794]
 [  0   0   0   0   0   0   0   0   0 883]
 [  0   0   0   0   0   0   0   0   0 841]
 [  0   0   0   0   0   0   0   0   0 845]]
Training model with data size at: 0.003%.
Training finished.
Statistics summary
-------------------------------------------------------------

Accuracy: 0.2755952380952381%

Confusion matrix:
[[818   0   0   0   0   0   3   0   0  12]
 [  3   0   0   0   0   0  16   0   0 939]
 [423   0   0   0   0   0 234   0   0 158]
 [610   0   0   0   0   0  25   0   0 244]
 [ 30   0   